In [1]:
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense

from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme
import numpy as np
from quant_rotor.models.sparse.support_ham import build_V_in_p
import matplotlib.pyplot as plt
from quant_rotor.models.dense.density_matrix import density_matrix_1
import quant_rotor.models.dense.thermofield_boltz_funcs as bz
import opt_einsum as oe

import scipy.sparse as sp

In [2]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=np.inf)

# Check TF

In [58]:
site = 3
state = 3
g = 0.1

In [59]:
H, K, V = hamiltonian_dense(state, site, g)

In [60]:
state_2 = state**2

In [61]:
U, _ = bz.thermofield_change_of_basis(K)

I = np.eye(state)

K_prim = oe.contract('pq,mw->pmqw', K, I,  optimize='optimal').reshape(state_2, state_2)

K_tilda = U.T @ K_prim @ U

V_tensor = V.reshape(state, state, state, state)

V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U.T, U.T, V_grouped, U, U, optimize='optimal').reshape(state_2**2, state_2**2)

In [62]:
V_tilda.shape

(81, 81)

In [ ]:
H_TF_prim, _, _ = hamiltonian_dense(state**2, site, g, K_import=K_prim, V_import=V_prim, Import=True)
H_TF, _, _ = hamiltonian_dense(state**2, site, g, K_import=K_prim, V_import=V_prim, Import=True)

In [64]:
np.diag(H_TF.real)

array([2.     , 2.33333, 1.66667, 1.33333, 1.33333, 2.33333, 2.33333, 2.33333, 2.33333, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 1.66667, 2.     , 1.33333, 1.     , 1.     , 2.     , 2.     , 2.     , 2.     , 1.33333, 1.66667, 1.     , 0.66667, 0.66667, 1.66667, 1.66667, 1.66667, 1.66667, 1.33333, 1.66667, 1.     , 0.66667, 0.66667, 1.66667, 1.66667, 1.66667, 1.66667, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 2.33333, 2.66667, 2.     , 1.66667, 1.66667, 2.66667, 2.66667, 2.66667, 2.66667, 2.66667, 3.     , 2.33333, 2.     , 2.     , 3.     , 3.     , 3.     , 3.     , 2.     , 2.33333, 1.66667, 1.33333, 1.33333, 2.33333, 2.33333, 2.33333, 2.33333, 1.66667, 2.     , 1.3

In [80]:
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)
eig_val, eig_vec = np.linalg.eigh(H)

In [81]:
index_OR = np.argsort(-eig_val)

In [82]:
eig_val[index_OR]

array([ 3.00016,  3.00009,  3.00005,  3.00005,  3.00001,  3.00001,  3.     ,  3.     ,  2.01323,  2.01323,  2.01004,  2.01   ,  2.01   ,  2.005  ,  1.995  ,  1.995  ,  1.995  ,  1.98677,  1.98677,  1.98015,  1.00999,  1.00999,  1.00991,  0.99495,  0.99495,  0.97984, -0.00019])

In [83]:
index_TF = np.argsort(-eig_val_TF)

In [84]:
eig_val_TF[index_TF[3]]

np.float64(3.00016087827633)

In [85]:
psi_vec = eig_vec_TF[index_TF[9]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [86]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [87]:
eig_val_D[index]

array([0.33543, 0.20193, 0.18418, 0.11132, 0.0625 , 0.05772, 0.0469 , 0.00001, 0.00001])

# Sparse

In [73]:
site = 3
state = 3
g = 0.9

In [74]:
H, K, V = hamiltonian_general_sparse(state, site, g)

K = K.toarray()
V = V.toarray()

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/sparse/_sputils.py:147: ComplexWarning: Casting complex values to real discards the imaginary part
  data = np.array(obj, dtype=dtype, copy=copy)


In [75]:
U, _ = bz.thermofield_change_of_basis(K)

K_tilda = bz.H_tilde_maker(K)

V_tensor = V.reshape(state, state, state, state)

I = np.eye(state)
V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U, U, V_grouped, U, U, optimize='optimal')

In [76]:
state_2 = state**2

K_tilda_s = sp.csr_matrix(K_tilda)
V_tilda_s = sp.csr_matrix(V_tilda.reshape(state_2**2, state_2**2))

In [77]:
H_TF, _, _ = hamiltonian_sparse(state_2, site, g, K_import=K_tilda, V_import=V_tilda)

In [78]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

NameError: name 'rho_site_0' is not defined

In [ ]:
_, psi_vec = sp.linalg.eigsh(H_TF, k=1, which='SA', tol=1e-19, maxiter=2000)

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [ ]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [ ]:
eig_val_D[index[:2]]

array([0.62560978, 0.36452163])

# Dense

In [39]:
site = 3
state = 3
g = 0.9

In [40]:
H, K, V = hamiltonian_general_dense(state, site, g)

In [41]:
H.shape

(27, 27)

In [42]:
U, _ = bz.thermofield_change_of_basis(K)

K_tilda = bz.H_tilde_maker(K)

V_tensor = V.reshape(state, state, state, state)

I = np.eye(state)
V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U.T, U.T, V_grouped, U, U, optimize='optimal').reshape(state_2**2, state_2**2)

In [43]:
state_2 = state**2

In [44]:
H_TF, _, _ = hamiltonian_dense(state_2, site, g, K_import=K_tilda, V_import=V_tilda, Import=True)

In [45]:
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)

In [46]:
index = np.argsort(-eig_val_TF)
psi_vec = eig_vec_TF[index[0]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [47]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [48]:
eig_val_D[index]

array([ 0.75183,  0.18858,  0.05959,  0.     ,  0.     , -0.     , -0.     , -0.     , -0.     ])

In [49]:
matrix_p_to_NO_full[index[0]].real

array([ 0., -0.,  0., -0., -0.,  0., -0.,  1., -0.])